In [1]:
import pandas as pd
from pathlib import Path
from datetime import date

# Ruta del archivo (ajustar si usás otra fecha)
BASE_PATH = Path("..").resolve().parent
FECHA = date.today().isoformat()
BT_FILE = BASE_PATH / "reports" / "backtesting" / f"bt_heuristicas_{FECHA}.csv"

# Cargar archivo
df = pd.read_csv(BT_FILE)
df["fecha_entry"] = pd.to_datetime(df["fecha_entry"], errors="coerce")
df["fecha_exit"] = pd.to_datetime(df["fecha_exit"], errors="coerce")

print(f"Total operaciones: {len(df)}")
print("Columnas:", list(df.columns))
df.head(3)


Total operaciones: 5128411
Columnas: ['simbolo', 'estrategia', 'fecha_entry', 'fecha_exit', 'signal', 'entry', 'exit', 'retorno', 'tp_hit', 'sl_hit']


,simbolo,estrategia,fecha_entry,fecha_exit,signal,entry,exit,retorno,tp_hit,sl_hit
0,A,gap_open_strategy,2005-08-15,2005-08-24,buy,21.69528,21.56652,-0.0059,False,False
1,A,gap_open_strategy,2005-10-14,2005-10-25,buy,22.12446,22.30329,0.0081,False,False
2,A,gap_open_strategy,2005-11-15,2005-11-25,buy,24.67811,25.52933,0.0345,False,False


In [ ]:
resumen_estrategia = (
    df.groupby("estrategia")
    .agg(
        total_op=("retorno", "count"),
        winrate=("retorno", lambda x: round((x > 0).sum() / len(x), 2)),
        retorno_total=("retorno", "sum"),
        sharpe=("retorno", lambda x: round(x.mean() / x.std(), 2) if x.std() > 0 else 0),
        profit_factor=("retorno", lambda x: round(x[x > 0].sum() / abs(x[x < 0].sum()), 2) if x[x < 0].sum() != 0 else float("inf"))
    )
    .sort_values("retorno_total", ascending=False)
)

print("Resumen por estrategia:")
display(resumen_estrategia)


In [ ]:
resumen_simbolo = (
    df.groupby("simbolo")
    .agg(
        total_op=("retorno", "count"),
        winrate=("retorno", lambda x: round((x > 0).sum() / len(x), 2)),
        retorno_total=("retorno", "sum"),
        sharpe=("retorno", lambda x: round(x.mean() / x.std(), 2) if x.std() > 0 else 0),
        profit_factor=("retorno", lambda x: round(x[x > 0].sum() / abs(x[x < 0].sum()), 2) if x[x < 0].sum() != 0 else float("inf"))
    )
    .sort_values("retorno_total", ascending=False)
)

print("Resumen por simbolo:")
display(resumen_simbolo)


In [ ]:
top_combinaciones = (
    df.groupby(["estrategia", "simbolo"])
    .agg(
        total_op=("retorno", "count"),
        retorno_total=("retorno", "sum"),
        winrate=("retorno", lambda x: round((x > 0).sum() / len(x), 2)),
        sharpe=("retorno", lambda x: round(x.mean() / x.std(), 2) if x.std() > 0 else 0)
    )
    .reset_index()
    .sort_values("retorno_total", ascending=False)
)

print("Top combinaciones estrategia + simbolo:")
display(top_combinaciones.head(20))


In [ ]:
print("Resumen de métricas globales en top_combinaciones:\n")
print("Total combinaciones:", len(top_combinaciones))
print("Sharpe ratio - min:", round(top_combinaciones["sharpe"].min(), 2), "| max:", round(top_combinaciones["sharpe"].max(), 2))
print("Winrate - min:", round(top_combinaciones["winrate"].min(), 2), "| max:", round(top_combinaciones["winrate"].max(), 2))
print("Operaciones - min:", top_combinaciones["total_op"].min(), "| max:", top_combinaciones["total_op"].max())
print("Retorno total - min:", round(top_combinaciones["retorno_total"].min(), 2), "| max:", round(top_combinaciones["retorno_total"].max(), 2))

In [ ]:
# Mostrar top 10 por retorno
print("\nTop 10 por retorno total:")
display(top_combinaciones.sort_values("retorno_total", ascending=False).head(20))

In [ ]:
# === Top combinaciones por winrate ===
top_winrate = (
    top_combinaciones
    .sort_values("winrate", ascending=False)
    .head(20)  # puedes ajustar a 10, 50, etc
)

print("Top combinaciones por winrate:")
display(top_winrate[["estrategia", "simbolo", "total_op", "retorno_total", "winrate", "sharpe"]])


In [ ]:
# === Top combinaciones por winrate ===
top_winrate = (
    top_combinaciones
    .sort_values("winrate", ascending=False)
    .head(20)  # puedes ajustar a 10, 50, etc
)

print("Top combinaciones por winrate:")
display(top_winrate[["estrategia", "simbolo", "total_op", "retorno_total", "winrate", "sharpe"]])


In [ ]:
# Winrate alto y al menos 50 operaciones
top_winrate_filtrado = top_combinaciones[top_combinaciones["total_op"] >= 100] \
    .sort_values("winrate", ascending=False).head(20)

display(top_winrate_filtrado)


In [ ]:
# === Top 20 combinaciones por retorno_total ===
top_rentables = (
    top_combinaciones
    .sort_values("retorno_total", ascending=False)
    .head(20)
)

print("Top 20 combinaciones por retorno total:")
display(top_rentables[["estrategia", "simbolo", "total_op", "retorno_total", "winrate", "sharpe"]])


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(top_rentables["simbolo"] + " | " + top_rentables["estrategia"], top_rentables["retorno_total"])
plt.xlabel("Retorno total")
plt.title("Top combinaciones por retorno total")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
min_op = 100  # puedes ajustar este umbral
top_rentables_filtradas = (
    top_combinaciones[top_combinaciones["total_op"] >= min_op]
    .sort_values("retorno_total", ascending=False)
    .head(20)
)

print(f"Top 20 combinaciones rentables con al menos {min_op} operaciones:")
display(top_rentables_filtradas)


In [ ]:
import pandas as pd
from pathlib import Path
from datetime import date

# === Rutas
BASE_PATH = Path("..").resolve().parent
BT_DIR = BASE_PATH / "reports" / "backtesting"
FECHA = date.today().isoformat()
ARCHIVO = BT_DIR / f"bt_heuristicas_{FECHA}.csv"

# === Cargar archivo
df_bt = pd.read_csv(ARCHIVO, parse_dates=["fecha_entry", "fecha_exit"])
print(f"Total operaciones: {len(df_bt)}")
display(df_bt.head())



In [ ]:
print("Columnas:", list(df_bt.columns))
print("\nTipos:")
print(df_bt.dtypes)

print("\nFechas:")
print("Min:", df_bt["fecha_entry"].min(), "→ Max:", df_bt["fecha_exit"].max())

print("\nEstrategias únicas:", df_bt["estrategia"].nunique())
print("Símbolos únicos:", df_bt["simbolo"].nunique())
print("Signals:", df_bt["signal"].unique())


In [ ]:
top_combinaciones = (
    df_bt.groupby(["estrategia", "simbolo"])
    .agg(
        total_op=("retorno", "count"),
        retorno_total=("retorno", "sum"),
        winrate=("retorno", lambda x: round((x > 0).sum() / len(x), 2)),
        sharpe=("retorno", lambda x: round(x.mean() / x.std(), 2) if x.std() > 0 else 0)
    )
    .reset_index()
    .sort_values("retorno_total", ascending=False)
)
display(top_combinaciones.head(10))


In [ ]:
min_op = 100
min_sharpe = 0.3
min_winrate = 0.6

ganadoras = top_combinaciones[
    (top_combinaciones["total_op"] >= min_op) &
    (top_combinaciones["sharpe"] >= min_sharpe) &
    (top_combinaciones["winrate"] >= min_winrate)
].sort_values("retorno_total", ascending=False)

print(f"\nCombinaciones ganadoras (op≥{min_op}, Sharpe≥{min_sharpe}, winrate≥{min_winrate}):")
display(ganadoras)


In [ ]:
top_rentables = top_combinaciones.sort_values("retorno_total", ascending=False).head(20)
display(top_rentables)


In [ ]:
top_sharpe = top_combinaciones.sort_values("sharpe", ascending=False).head(20)
display(top_sharpe)


In [ ]:
top_winrate = top_combinaciones.sort_values("winrate", ascending=False).head(20)
display(top_winrate)


In [ ]:
from pathlib import Path
import json

# === Ruta de salida
EXPORT_PATH = BASE_PATH / "config" / "json" / "combinaciones_filtradas.json"
EXPORT_PATH.parent.mkdir(parents=True, exist_ok=True)

# === Preparar estructura de salida
lista_final = [
    {"simbolo": row["simbolo"], "estrategia": row["estrategia"]}
    for _, row in ganadoras.iterrows()
]

# === Guardar como JSON
with open(EXPORT_PATH, "w") as f:
    json.dump({"combinaciones": lista_final}, f, indent=2)

print(f"Combinaciones ganadoras exportadas a: {EXPORT_PATH}")
print(f"Total: {len(lista_final)} combinaciones")


In [ ]:
# === Filtros personalizados ===
min_op = 50
min_sharpe = 0.5
min_winrate = 0.50

top_filtrado = (
    top_combinaciones[
        (top_combinaciones["total_op"] >= min_op) &
        (top_combinaciones["sharpe"] > min_sharpe) &
        (top_combinaciones["winrate"] >= min_winrate)
    ]
    .sort_values("retorno_total", ascending=False)
    .head(30)
)

print(f"Combinaciones con op≥{min_op}, Sharpe>{min_sharpe}, winrate≥{min_winrate} (top 30 por retorno total):")
display(top_filtrado)


In [ ]:
len(top_filtrado)

In [ ]:
# === Extraer listas únicas ===
lista_simbolos = sorted(top_filtrado["simbolo"].unique().tolist())
lista_estrategias = sorted(top_filtrado["estrategia"].unique().tolist())

print(f"Simbolos seleccionados ({len(lista_simbolos)}):\n{lista_simbolos}")
print(f"\nEstrategias seleccionadas ({len(lista_estrategias)}):\n{lista_estrategias}")


In [ ]:
import json

# === Directorio de salida (ajustable si lo necesitas) ===
OUTPUT_DIR = Path("notebooks/test/config/filtrados")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# === Extraer listas únicas ===
lista_simbolos = sorted(top_filtrado["simbolo"].unique().tolist())
lista_estrategias = sorted(top_filtrado["estrategia"].unique().tolist())

# === Mostrar en consola ===
print(f"Simbolos seleccionados ({len(lista_simbolos)}):\n{lista_simbolos}")
print(f"\nEstrategias seleccionadas ({len(lista_estrategias)}):\n{lista_estrategias}")

# === Guardar como JSON ===
with open(OUTPUT_DIR / "simbolos_filtrados.json", "w") as f:
    json.dump({"simbolos": lista_simbolos}, f, indent=2)

with open(OUTPUT_DIR / "estrategias_filtradas.json", "w") as f:
    json.dump({"estrategias": lista_estrategias}, f, indent=2)

# === Guardar como TXT simple ===
with open(OUTPUT_DIR / "simbolos_filtrados.txt", "w") as f:
    f.write("\n".join(lista_simbolos))

with open(OUTPUT_DIR / "estrategias_filtradas.txt", "w") as f:
    f.write("\n".join(lista_estrategias))

print("\nArchivos guardados en:", OUTPUT_DIR.resolve())
